<a href="https://colab.research.google.com/github/artms-18/tensorflow_fundamentals/blob/main/Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-088e628f-f849-a960-938f-f712df3ee177)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-05-18 12:13:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-18 12:13:40 (94.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataaset

The dataset we're going to be using is Kaggle's introduction to NLP dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2021-05-18 12:13:42--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.128, 74.125.199.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-05-18 12:13:43 (99.5 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualie our text samples, we first have to read them in, one way to do so would be to use Python: 

Another way to do this is to use pandas.

(If text dataset is very large, cannot ise pandas (will tkaes of to much RAM).



In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle training dataframe

train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples

import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples

for row in train_df_shuffled[['text', 'target']][random_index: random_index + 5].itertuples():
  _, text, target = row #gets rid of index
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster
Text:
@danagould @WaynesterAtl I agree with background checks. I just think guns or weapons in general are the great equalizer.

---

Target: 0 (not real disaster
Text:
Drake is really body bagging meek

---

Target: 0 (not real disaster
Text:
Ignition Knock (Detonation) Sensor-Senso Standard fits 03-08 Mazda 6 3.0L-V6 http://t.co/c8UXkIzwM6 http://t.co/SNxgH9R16u

---

Target: 0 (not real disaster
Text:
she's a natural disaster she's the last of the American girls ??

---

Target: 1 (real disaster)
Text:
... The pain of those seconds must have been awful as her heart burst and her lungs collapsed and there was no air and...

---



### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split top split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1, random_state = 42)

In [13]:
# Check the lengths

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a midek is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of  token( a toke could be a word or a character) to number
* embedding - create a matrix of feature vectors for each token the size of the feature vector can be defined and this embedding can be learned

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens = None, # how many words in the vocab (automatically add <OOV>)
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace", 
                                    ngrams = None,
                                    output_mode = "int", #how to map tokens to numbers
                                    output_sequence_length = None, #pads each sequence to the logest sequence
                                    pad_to_max_tokens = True)

In [17]:
# Find the average number of tokens (words) in the training tweets

round(sum(len(i.split()) for i in train_sentences)) / len(train_sentences)

14.901036345059115

In [18]:
# Setup text vectorization variables

max_vocab_length = 10000 #max num of words to have in our vocabulary
max_length = 15 # max length our seuqences will be (how many words in a tweet, amodel would see)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length, output_mode= 'int', output_sequence_length = max_length)

In [19]:
#fit the text vectorier to the training text

text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it

sample_sentence = "Theres a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize iot

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Ashley and I on going to hurricane harbor Friday. ?? http://t.co/ScEfPFvAEU

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[6186,    7,    8,   11,  104,    5,  346, 2968, 1208,    1,    0,
           0,    0,    0,    0]])>

In [22]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get all of the unique words in the training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most comon words: {top_5_words}")

Number of words in vocab: 10000
5 most comon words: ['', '[UNK]', 'the', 'a', 'in']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use tensoflows embedding layer

The parameters we actually care about:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vecotr 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [23]:
from tensorflow.keras import layers

# in keras, wordks best when numbers are divisible by eight

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length
                             )
embedding

In [24]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn into dense vecotrs of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 70 years after : Hiroshima and Nagasaki - consequences of a nuclear detonation @ICRC http://t.co/BKh7Z6CWWl

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03621849,  0.01880667, -0.01181567, ..., -0.02582639,
         -0.0043739 , -0.01223137],
        [ 0.04926355, -0.02206566,  0.00567269, ..., -0.03911684,
         -0.02108531, -0.03856306],
        [ 0.01454754, -0.01602609, -0.00841321, ...,  0.04051122,
         -0.04294854,  0.02689974],
        ...,
        [-0.01167753,  0.03712274, -0.0167491 , ..., -0.00761473,
          0.0063177 , -0.01721437],
        [ 0.01053485,  0.04474126, -0.01283311, ...,  0.00386763,
          0.0018299 ,  0.04687903],
        [ 0.01053485,  0.04474126, -0.01283311, ...,  0.00386763,
          0.0018299 ,  0.04687903]]], dtype=float32)>

In [25]:
# Chekc out a single token's embedding

sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03621849,  0.01880667, -0.01181567, -0.01712491, -0.03947524,
        -0.00829355, -0.03653984,  0.01443431,  0.04993571, -0.01821802,
        -0.03725889, -0.01276745, -0.0044008 , -0.02598683, -0.02722323,
        -0.01779775,  0.03709911, -0.0398448 ,  0.03826887, -0.0030322 ,
        -0.04028846, -0.0031046 , -0.00296867, -0.016105  , -0.02523552,
        -0.00194386, -0.00882415, -0.03561758,  0.0376277 , -0.00574417,
        -0.00131885, -0.01588482,  0.00262376,  0.04429774,  0.01283542,
        -0.04933331,  0.03343562, -0.01049539, -0.00618013, -0.00108596,
         0.00719047,  0.02158047,  0.0475513 ,  0.02934648, -0.04543207,
        -0.01349826, -0.0180971 ,  0.04938115,  0.00943778,  0.00664321,
        -0.04818244, -0.00030422, -0.04670403, -0.04184397,  0.03788263,
         0.02427844,  0.03281391, -0.02984818, -0.04315795, -0.0175156 ,
        -0.02654743, -0.02580723,  0.02045033,  0.00717016,  0.01294817,
  

## Modelling a text dataset (running a series of experiments)

Now we're got a wat to turn our text srequences into numbers, it's time to start building a series of modelling experiments, start from a baseline and move on from there 

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neual network (dense model)
* Model 2: GRU model (RNN)
* Model 4: Bidirectional-LSTM
* Model 5: 1d Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Geature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Compile a model
* Fit the model

### Model 0: Getting a baseline

As with all machine learning modelling experiments it's important to create a baseline model so u got a benchmark for future experiments to build upon

To create our baseline, we'll use SKlearns Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers

> **Note:** It's common practive to use non-DL algorithms as a baseline because of their speed, then later using DL to improve

In [26]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data

model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
# Evaluate our baseline model 

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate model's predictin with different methods every time, but this could be cumbersome and could easily be fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [29]:
# Function to evaluate: accuracty, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calulates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall ,and f01 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [30]:
baseline_results = calculate_results(y_true = val_labels, y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [31]:
# Create a tensorboard callback (need to create a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs

SAVE_DIR = "model_logs"

In [32]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = tf.string) # inputs are 1-dimensional
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized imputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
#x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x) # Create the output layer, want binary output so use sigmoid

model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile mode

model_1.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [35]:
# Fit the mode
model_1_history = model_1.fit(x = train_sentences, y = train_labels, epochs = 15, validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210518-121349
Epoch 1/15
215/215 [==============================] - 7s 17ms/step - loss: 0.6528 - accuracy: 0.6421 - val_loss: 0.5367 - val_accuracy: 0.7572
Epoch 2/15
215/215 [==============================] - 3s 13ms/step - loss: 0.4544 - accuracy: 0.8168 - val_loss: 0.4673 - val_accuracy: 0.7861
Epoch 3/15
215/215 [==============================] - 3s 13ms/step - loss: 0.3588 - accuracy: 0.8534 - val_loss: 0.4578 - val_accuracy: 0.7848
Epoch 4/15
215/215 [==============================] - 3s 13ms/step - loss: 0.2787 - accuracy: 0.8945 - val_loss: 0.4611 - val_accuracy: 0.7835
Epoch 5/15
215/215 [==============================] - 3s 13ms/step - loss: 0.2349 - accuracy: 0.9161 - val_loss: 0.4775 - val_accuracy: 0.7861
Epoch 6/15
215/215 [==============================] - 3s 13ms/step - loss: 0.1996 - accuracy: 0.9289 - val_loss: 0.4980 - val_accuracy: 0.7795
Epoch 7/15
215/215 [==============================] - 3s 13ms/step -

In [36]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.7683 - accuracy: 0.7743


[0.7683051228523254, 0.7742782235145569]

In [37]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [38]:
# look at a single prediction

model_1_pred_probs[0] # anything below 0.5 = disaster, abouve = non-=disaster

array([0.70264345], dtype=float32)

In [39]:
# look at the first 10 predictions

model_1_pred_probs[:10]

array([[7.0264345e-01],
       [8.7971568e-01],
       [9.9937743e-01],
       [1.1590025e-01],
       [9.0633775e-04],
       [9.9145621e-01],
       [8.1805092e-01],
       [9.9997759e-01],
       [9.9996245e-01],
       [7.2765166e-01]], dtype=float32)

In [40]:
# Convert model prediction probablilities to label format

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# calculate our model_1 results

model_1_results = calculate_results(y_true = val_labels, y_pred = model_1_preds)

model_1_results

{'accuracy': 77.42782152230971,
 'f1': 0.7732392305682255,
 'precision': 0.7743239391456158,
 'recall': 0.7742782152230971}

In [42]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [43]:
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

# Visualizing learned embeddings


In [44]:
# get the vocab from the text vecotrization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Get the wieght matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epocks)

embed_weights = model_1.get_layer("embedding").get_weights()[0]

print(embed_weights.shape) # same size as vocab size and embedding dim (output dim of our embedding layer)

(10000, 128)


In [47]:
embed_weights

array([[-0.00363728,  0.03172309, -0.02543144, ..., -0.009585  ,
         0.0149083 ,  0.05956154],
       [-0.00421884,  0.04522499, -0.00732461, ...,  0.00134055,
        -0.00247424, -0.02557608],
       [ 0.02357583, -0.05212818,  0.01887457, ...,  0.00791396,
        -0.00856107,  0.01003446],
       ...,
       [ 0.04136404, -0.02715068,  0.02548103, ...,  0.0278498 ,
        -0.00665744,  0.0280537 ],
       [-0.07654047, -0.02104904, -0.00813236, ..., -0.06453947,
         0.02045591,  0.06372694],
       [-0.19820657, -0.18752962, -0.19574657, ..., -0.20531216,
         0.16464043,  0.12085287]], dtype=float32)

In [48]:
# Create embedding files

import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  #if index == 0:
   # continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# Download files from Colab to upload to projector

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)

RNN's are usefull for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the respresentation of a later input.

### Model 2: LSTM

LSTM = ling short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

'''
Input (text) -> Tokenize -> EMbedding -> Layers (RNN's/dense) -> Output (label probability)
'''

In [50]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
#x = layers.LSTM(64, return_sequences = True)(x)
print(x.shape)
x = layers.LSTM(64)(x) # when you're stacking RNN cells together, you need to return_sequences = true
print(x.shape)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 128)
(None, 64)


In [51]:
# Compile the model

model_2.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [52]:
model_2_history = model_2.fit(train_sentences, train_labels, epochs = 5, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210518-121437
Epoch 1/5
215/215 [==============================] - 34s 21ms/step - loss: 0.2136 - accuracy: 0.9414 - val_loss: 0.8598 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0636 - accuracy: 0.9761 - val_loss: 0.9347 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0521 - accuracy: 0.9810 - val_loss: 1.0138 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0521 - accuracy: 0.9819 - val_loss: 1.1582 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0478 - accuracy: 0.9780 - val_loss: 1.1965 - val_accuracy: 0.7598


In [53]:
# Make predictions with LSTM model

model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


array([[4.1672204e-02],
       [8.2745004e-01],
       [9.9969828e-01],
       [9.8737873e-02],
       [2.1172622e-04],
       [9.9967933e-01],
       [7.6788896e-01],
       [9.9989116e-01],
       [9.9987888e-01],
       [9.9961156e-01]], dtype=float32)

In [54]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
model_2_results = calculate_results(y_true = val_labels, y_pred = model_2_preds)
model_2_results

{'accuracy': 75.98425196850394,
 'f1': 0.7588879536706105,
 'precision': 0.7595905514032232,
 'recall': 0.7598425196850394}

### Model 3: GRU

Another poppular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [56]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#x = layers.LSTM(64, return_sequences = True)(x)
#print(x.shape)
#x = layers.GRU(64)(x) # make sure return seuqences DOES NO equal true OR add a global average pooling layer instead
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)


model_3 = tf.keras.Model(inputs, outputs)

In [57]:
 model_3.summary(
     
 )

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [58]:
# COmpile the model

model_3.compile(loss = "binary_crossentropy", optimizer= tf.keras.optimizers.Adam(), metrics = ["accuracy"])


In [59]:
# Fit

model_3_history = model_3.fit(train_sentences,
                              train_labels, epochs = 10,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20210518-121526
Epoch 1/10
215/215 [==============================] - 6s 19ms/step - loss: 0.2518 - accuracy: 0.8822 - val_loss: 0.9632 - val_accuracy: 0.7664
Epoch 2/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0570 - accuracy: 0.9772 - val_loss: 1.0032 - val_accuracy: 0.7612
Epoch 3/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0519 - accuracy: 0.9778 - val_loss: 1.0750 - val_accuracy: 0.7467
Epoch 4/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0478 - accuracy: 0.9792 - val_loss: 1.0048 - val_accuracy: 0.7612
Epoch 5/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0402 - accuracy: 0.9814 - val_loss: 1.1274 - val_accuracy: 0.7664
Epoch 6/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0383 - accuracy: 0.9810 - val_loss: 1.5110 - val_accuracy: 0.7703
Epoch 7/10
215/215 [==============================] - 3s 14ms/step - l

In [60]:
# Make some predictions with GRu model

model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[3.6164769e-03],
       [7.5106859e-01],
       [9.9979275e-01],
       [1.7636587e-01],
       [2.0929934e-05],
       [9.9817789e-01],
       [8.3907974e-01],
       [9.9993324e-01],
       [9.9993587e-01],
       [9.9878210e-01]], dtype=float32)

In [61]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
# Calculate model 3 results

model_3_results = calculate_results(y_true = val_labels, y_pred = model_3_preds)
model_3_results

{'accuracy': 75.45931758530183,
 'f1': 0.7531886844350836,
 'precision': 0.7546642240189775,
 'recall': 0.7545931758530183}

### Model 4: Bidirectional RNN

ormal RNN's go from left to right (just like you'd read an English sentence) however, a bidiractional RNN goes from right to left as well as left to right.

In [63]:
# Build a dirirectional RNN in Tensorflow

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
#print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_bidirectional")

(None, 128)


In [64]:
# Get a summary

model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               74496     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,354,625
Trainable params: 1,354,625
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile model

model_4.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

In [66]:
model_4_history = model_4.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                                                                                                                 "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210518-121601
Epoch 1/5
215/215 [==============================] - 8s 23ms/step - loss: 0.1948 - accuracy: 0.9432 - val_loss: 0.9677 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0432 - accuracy: 0.9843 - val_loss: 1.2405 - val_accuracy: 0.7533
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0335 - accuracy: 0.9833 - val_loss: 1.1145 - val_accuracy: 0.7520
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0361 - accuracy: 0.9829 - val_loss: 1.3941 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0361 - accuracy: 0.9824 - val_loss: 1.4721 - val_accuracy: 0.7546


In [67]:
# Make predictions with out bidiractional ,odel
 
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[2.8678824e-03],
       [8.6277127e-01],
       [9.9991465e-01],
       [1.3207206e-01],
       [1.0566581e-05],
       [9.9955863e-01],
       [9.9727219e-01],
       [9.9999249e-01],
       [9.9996769e-01],
       [9.9802905e-01]], dtype=float32)

In [68]:
# Convert

model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
# Calculate the results of our bidiractional model

model_4_results = calculate_results(y_true = val_labels, y_pred = model_4_pred )
model_4_results

{'accuracy': 75.45931758530183,
 'f1': 0.7535158648005912,
 'precision': 0.7543596436293261,
 'recall': 0.7545931758530183}

### Model 5: Conv1D 

We've used CNNs for images but images are typically 2D (hegith x width)...however, our text data is 1d.

The typical structure of a Conv1D model for sequen ces (in our case, text):

```
Inputs(test -> Tokenization -> Embedding -> Cpmv1D -> Pool -> Outputs (class probs)
```

In [70]:
from tensorflow.keras import layers

input = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.Conv1D(32, 6, activation = 'relu')(x) #kernal size is reffered to as n-gram side (strides rn is 1, but can change to hop) #padding = 'valid' (some words mised) output is smaller 'same' = smae size
x = layers.GlobalMaxPooling1D()(x)
output = layers.Dense(1, activation = "sigmoid")(x)

model_5 = tf.keras.Model(input, output, name = "model_5_conv")


In [71]:
model_5.summary()

Model: "model_5_conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 10, 32)            24608     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,304,641
Trainable params: 1,304,641
Non-trainable params: 0
____________________________________________

In [72]:
# Compiling the model

model_5.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])



In [73]:
# Fitting the model

model_5_history = model_5.fit(train_sentences, train_labels, epochs = 10, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_5_conv")])

Saving TensorBoard log files to: model_logs/model_5_conv/20210518-121624
Epoch 1/10
215/215 [==============================] - 5s 18ms/step - loss: 0.1694 - accuracy: 0.9473 - val_loss: 1.0373 - val_accuracy: 0.7546
Epoch 2/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0564 - accuracy: 0.9792 - val_loss: 1.1870 - val_accuracy: 0.7572
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0528 - accuracy: 0.9778 - val_loss: 1.2771 - val_accuracy: 0.7585
Epoch 4/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0434 - accuracy: 0.9822 - val_loss: 1.2645 - val_accuracy: 0.7546
Epoch 5/10
215/215 [==============================] - 3s 13ms/step - loss: 0.0393 - accuracy: 0.9828 - val_loss: 1.3628 - val_accuracy: 0.7585
Epoch 6/10
215/215 [==============================] - 3s 13ms/step - loss: 0.0356 - accuracy: 0.9852 - val_loss: 1.4187 - val_accuracy: 0.7480
Epoch 7/10
215/215 [==============================] - 3s 14ms/step - 

In [81]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64, kernel_size = 5, activation = "relu")(x)
x = layers.GlobalMaxPool1D()(x)
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_6 = tf.keras.Model(inputs, outputs, name = "model_6_Conv1D")

model_6.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_6.summary()

Model: "model_6_Conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [87]:
# Fit the model

model_6_history = model_6.fit(train_sentences, train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20210518-122427
Epoch 1/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1213 - accuracy: 0.9600 - val_loss: 1.2058 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0464 - accuracy: 0.9782 - val_loss: 1.3217 - val_accuracy: 0.7493
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0372 - accuracy: 0.9842 - val_loss: 1.3357 - val_accuracy: 0.7467
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0321 - accuracy: 0.9847 - val_loss: 1.3629 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0332 - accuracy: 0.9849 - val_loss: 1.3252 - val_accuracy: 0.7520


In [88]:
# Make some prediction with our Conv1D model

model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [89]:
model_6_results = calculate_results(y_true = val_labels, y_pred = model_6_pred)
model_6_results

{'accuracy': 75.19685039370079,
 'f1': 0.7517482001130363,
 'precision': 0.751655404843956,
 'recall': 0.7519685039370079}

## Model6: Tensorflow Hub Pretrained Sentence Encoder